In [1]:
%%html
<style>
div.output_area pre 
{
    white-space: pre;
}

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
       .builder \
       .appName('aggregate_bookings_data') \
       .getOrCreate() 

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1661943132780_0017,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.sql("SHOW TABLES").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
| default|     tbl_bookings|      false|
| default|tbl_click_streams|      false|
+--------+-----------------+-----------+

In [4]:
bookings = spark.sql("SELECT * FROM TBL_BOOKINGS")
bookings.printSchema()
bookings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- booking_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- driver_id: integer (nullable = true)
 |-- customer_app_version: string (nullable = true)
 |-- customer_phone_os_version: string (nullable = true)
 |-- pickup_lat: float (nullable = true)
 |-- pickup_lon: float (nullable = true)
 |-- drop_lat: float (nullable = true)
 |-- drop_lon: float (nullable = true)
 |-- pickup_timestamp: timestamp (nullable = true)
 |-- drop_timestamp: timestamp (nullable = true)
 |-- trip_fare: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- cab_color: string (nullable = true)
 |-- cab_registration_no: string (nullable = true)
 |-- customer_rating_by_driver: integer (nullable = true)
 |-- rating_by_customer: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)

+------------+-----------+---------+--------------------+-------------------------+----------+----------+----------+------

In [5]:
from pyspark.sql.functions import col, to_date, count, sum as pyspark_sum

bookings_aggregated = bookings.withColumn("trip_date", to_date(col("pickup_timestamp"), "yyyy-MM-dd")) \
    .groupBy(col("trip_date")) \
    .agg(
        count("booking_id").alias("number_of_trips"),
        pyspark_sum("trip_fare").alias("total_trip_amount"),
        pyspark_sum("tip_amount").alias("total_tip_amount"),
        pyspark_sum("passenger_count").alias("total_passanger_counts")
        ) \
    .orderBy("trip_date") \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bookings_aggregated.show()
bookings_aggregated.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+-----------------+----------------+----------------------+
| trip_date|number_of_trips|total_trip_amount|total_tip_amount|total_passanger_counts|
+----------+---------------+-----------------+----------------+----------------------+
|2020-01-01|              1|            140.0|            59.0|                     3|
|2020-01-02|              3|           1502.0|            95.0|                     6|
|2020-01-03|              2|           1190.0|            11.0|                     8|
|2020-01-04|              2|            640.0|           123.0|                     4|
|2020-01-05|              2|            678.0|           134.0|                     7|
|2020-01-06|              3|           1328.0|           189.0|                     7|
|2020-01-07|              2|           1495.0|           148.0|                     5|
|2020-01-08|              4|           1899.0|           111.0|                    13|
|2020-01-09|              2|            778

In [7]:
bookings_aggregated.createOrReplaceTempView("VW_AGGREGATED_BOOKINGS")
spark.sql("CREATE TABLE IF NOT EXISTS TBL_AGGREGATED_BOOKINGS \
            AS SELECT * FROM VW_AGGREGATED_BOOKINGS");

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [8]:
spark.sql("SHOW TABLES").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|tbl_aggregated_bo...|      false|
| default|        tbl_bookings|      false|
| default|   tbl_click_streams|      false|
|        |vw_aggregated_boo...|       true|
+--------+--------------------+-----------+